### Calculate Top-k Accuracy (TTA20)

In [34]:
from rdkit import Chem
from neutral import NeutraliseCharges
from multiprocessing import Pool
from rdkit import RDLogger
import pandas as pd
from rdkit.Chem.MolStandardize import rdMolStandardize
RDLogger.DisableLog('rdApp.*')

def canonicalize_smiles(smiles):
    if len(smiles)==0:
        return ''
    mol = Chem.MolFromSmiles(smiles)
    # lfc = MolStandardize.fragment.LargestFragmentChooser()
    
    if mol is not None:
        # mol2 = lfc.choose(mol)
        smi2=Chem.MolToSmiles(mol, isomericSmiles=True)
        smi,_=NeutraliseCharges(smi2)
        return smi 
    else:
        return ''
# df['substrate']=df['substrate'].apply(canonicalize_smiles)

In [3]:
#把带多个的拆成单个

In [35]:
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import MolStandardize
def canonicalize_predict(smiles):
    if len(smiles)==0:
        return ''
    mol = Chem.MolFromSmiles(smiles)
    lfc = MolStandardize.fragment.LargestFragmentChooser()
    
    if mol is not None:
        mol2 = lfc.choose(mol)
        smi2=Chem.MolToSmiles(mol2, isomericSmiles=True)
        smi,_=NeutraliseCharges(smi2)
        return smi
    else:
        return ''

In [5]:
# sub=list(map(remove_leavegroup,true_reactseq_lis))

## label

In [36]:
import os
import pandas as pd
from e_smiles import get_e_smiles, merge_smiles, get_edit_from_e_smiles
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

def process_smiles(smiles):
    return merge_smiles(smiles)




In [37]:
def kekulize_smiles(smi):
    smi = smi.replace(' ','')
    mol = Chem.MolFromSmiles(smi)
    Chem.Kekulize(mol, clearAromaticFlags = True)
    smi = Chem.MolToSmiles(mol, canonical = False, kekuleSmiles = True)
    return smi

In [38]:
n_best = 10
substrate_lis = []
true_tgt_lis = []
with open('test_enzyme_argue10_input_group.txt') as f:
    for line in f.readlines():
        line = line.replace('\n','').replace(" ","")
        substrate_lis += [line] * n_best
        true_tgt_lis.append(line)

In [39]:
len(true_tgt_lis)

510

In [40]:
data=pd.read_csv("test_enzyme_argue10_add_smiles.tsv")

In [41]:
data

,enzyme,metabolite,smiles
0,UGT,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...,Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(...
1,CYP3A4,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O,Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12
2,CYP3A4,[~CH3]C1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)...,OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O
3,UGT1A9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...,Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(...
4,CYP2C9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O,Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12
...,...,...,...
5095,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[?C@]34OO[?C@](C...,CC1CCC2[C@@H](C)C(OC3OC(C(=O)O)C(O)C(O)C3O)OC3...
5096,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)...,CC1CCC2[C@@H](C)C(OC3OC(C(=O)O)C(O)C(O)C3O)OC3...
5097,UGT2B7,[C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)(...,CC1CCC2[C@@H](C)[C@@H](OC3OC(C(=O)O)C(O)C(O)C3...
5098,UGT2B7,[?C@@H]1([~OH])[C@H](C)[C@H]2[C@]34OO[C@](C)(O...,C[C@@H]1CC[C@H]2[C@@H](C)C(OC3OC(C(=O)O)[C@H](...


In [42]:
n_best=10

In [43]:
enzymes = data['enzyme'].fillna('').astype(str).tolist()
predict_cocan_smiles = data['smiles'].fillna('').tolist()

fold = 10
num = 51
n = n_best  
enzyme_prefixes = ['CYP', 'UGT', 'SULT', 'AKR', 'GST', 'CES', 'FMO', 'ALDH', 'NAT', 'COMT', 'AOX', 'XDH', 'EPHX', 'NQO', 'ADH']

# 计算分数
score_1 = []
for i in range(fold):
    for j in range(num):
        for k in range(1, n + 1):
            score = 1 / k**2
            score_1.append(score)

# 初始化
vote_score_lis = [-1] * len(enzymes)
id_list = []

# 计算 vote_score_lis
for j in range(num):
    enzyme_score_dic = {}
    enzyme_category_map = {}

    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            id_list.append(idx)
            
            enzyme = enzymes[idx]
            s = score_1[idx] if enzyme != '' else 0
            
            if enzyme == '':
                continue  # 跳过空值

            # 根据 enzyme 的前缀分类
            enzyme_category = None
            for prefix in enzyme_prefixes:
                if enzyme.startswith(prefix):
                    enzyme_category = prefix
                    break

            if enzyme_category is None:
                continue  # 如果没有匹配到大类，跳过
            
            # 大类和亚型分别累积打分
            key = enzyme_category
            subtype_key = enzyme

            # 大类累积打分
            if key not in enzyme_score_dic:
                enzyme_score_dic[key] = s
            else:
                enzyme_score_dic[key] += s

            # 亚型累积打分
            if subtype_key not in enzyme_category_map:
                enzyme_category_map[subtype_key] = s
            else:
                enzyme_category_map[subtype_key] += s

    # 更新 vote_score_lis
    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            enzyme = enzymes[idx]

            # 根据大类和亚型分开处理
            enzyme_category = None
            for prefix in enzyme_prefixes:
                if enzyme.startswith(prefix):
                    enzyme_category = prefix
                    break

            if enzyme_category is None:
                continue  # 如果没有匹配到大类，跳过

            # 更新总分
            vote_score_lis[idx] = enzyme_category_map.get(enzyme, -1)

# 创建最终的 pre_enzyme_category_50_lis 和 pre_enzyme_50_lis
pre_enzyme_category_50_lis = []
pre_enzyme_50_lis = []

for j in range(num):
    enzyme_g = []
    vote_score_g = []
    
    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            enzyme_g.append(enzymes[idx])
            vote_score_g.append(vote_score_lis[idx])
            
    zip_a_b = zip(enzyme_g, vote_score_g)
    sorted_zip = sorted(zip_a_b, key=lambda x: x[1], reverse=True)
    enzyme_g, vote_score_g = zip(*sorted_zip)
    
    pre_enzyme_category_50 = []
    pre_enzyme_50 = []
    
    seen_enzyme_category = set()
    seen_enzyme = set()
    
    for enzyme in enzyme_g:
        enzyme_category = None
        for prefix in enzyme_prefixes:
            if enzyme.startswith(prefix):
                enzyme_category = prefix
                break

        if enzyme_category is None:
            continue  # 如果没有匹配到大类，跳过

        # 先检查是否已经有该大类的条目
        if enzyme_category in seen_enzyme_category:
            # 如果大类已经存在，则仅保留带有亚型的酶
            if enzyme != enzyme_category and enzyme not in seen_enzyme and len(pre_enzyme_50) < 50:
                pre_enzyme_50.append(enzyme)
                seen_enzyme.add(enzyme)
        else:
            # 大类处理
            if len(pre_enzyme_category_50) < 50:
                pre_enzyme_category_50.append(enzyme_category)
                seen_enzyme_category.add(enzyme_category)

            # 亚型处理
            if enzyme != enzyme_category and len(pre_enzyme_50) < 50:
                pre_enzyme_50.append(enzyme)
                seen_enzyme.add(enzyme)
    
    pre_enzyme_category_50.extend([''] * (50 - len(pre_enzyme_category_50)))
    pre_enzyme_50.extend([''] * (50 - len(pre_enzyme_50)))

    pre_enzyme_category_50_lis.append(pre_enzyme_category_50)
    pre_enzyme_50_lis.append(pre_enzyme_50)


In [44]:
# 初始化最终的列表
combined_list = []

# 合并 pre_enzyme_category_50_lis 和 pre_enzyme_50_lis，只保留非空元素
for category_list, enzyme_list in zip(pre_enzyme_category_50_lis, pre_enzyme_50_lis):
    # 过滤掉空字符串
    filtered_category = [item for item in category_list if item != '']
    filtered_enzyme = [item for item in enzyme_list if item != '']
    
    # 合并非空元素
    combined = filtered_category + filtered_enzyme
    combined_list.append(combined)

# 查看合并后的列表
for combined in combined_list:
    print(combined)


['CYP', 'UGT', 'CYP3A4', 'CYP2C9', 'CYP2C19', 'CYP1A2', 'CYP2D6', 'CYP3A5', 'CYP1A1', 'CYP2B6', 'UGT1A9', 'UGT1A1', 'CYP2C8', 'UGT1A3']
['CYP', 'UGT', 'CYP3A4', 'UGT1A9', 'CYP2C8', 'UGT2B7', 'UGT1A1', 'UGT1A8', 'UGT2B4', 'CYP2D6', 'CYP2C19', 'CYP1A1']
['CYP', 'CYP2C19', 'CYP2D6', 'CYP3A4', 'CYP1A2', 'CYP2C9', 'CYP2C8', 'CYP1A1']
['CYP', 'UGT', 'CYP3A4', 'CYP2D6', 'CYP1A2', 'CYP1A1', 'CYP2C19', 'CYP2C8', 'CYP2C9', 'CYP3A5', 'UGT1A1']
['CYP', 'CYP3A4', 'CYP2D6', 'CYP1A1', 'CYP1A2']
['CYP', 'CYP2D6', 'CYP3A4', 'CYP1A2', 'CYP1A1', 'CYP2C8', 'CYP2C19', 'CYP2A6', 'CYP2C9', 'CYP2B6', 'CYP1A', 'CYP3A5']
['CYP', 'UGT', 'CYP3A4', 'UGT1A1', 'CYP1A2', 'UGT1A9', 'CYP2C19', 'UGT1A3', 'CYP2C9', 'UGT1A10', 'CYP2B6', 'CYP2D6', 'CYP1A1', 'CYP2A6', 'CYP3A5']
['CYP', 'CYP3A4', 'CYP1A1', 'CYP1A2', 'CYP2D6', 'CYP2C8', 'CYP2C19', 'CYP3A5', 'CYP2C9', 'CYP1B1']
['CYP', 'CYP3A4', 'CYP1A1', 'CYP1A2', 'CYP3A5', 'CYP2D6', 'CYP2C9', 'CYP2C19']
['CYP', 'CYP2C19', 'CYP1A2', 'CYP2B6', 'CYP3A4', 'CYP2C8', 'CYP1A1', 'CY

In [45]:
combined_list

[['CYP',
  'UGT',
  'CYP3A4',
  'CYP2C9',
  'CYP2C19',
  'CYP1A2',
  'CYP2D6',
  'CYP3A5',
  'CYP1A1',
  'CYP2B6',
  'UGT1A9',
  'UGT1A1',
  'CYP2C8',
  'UGT1A3'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'UGT1A9',
  'CYP2C8',
  'UGT2B7',
  'UGT1A1',
  'UGT1A8',
  'UGT2B4',
  'CYP2D6',
  'CYP2C19',
  'CYP1A1'],
 ['CYP',
  'CYP2C19',
  'CYP2D6',
  'CYP3A4',
  'CYP1A2',
  'CYP2C9',
  'CYP2C8',
  'CYP1A1'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'CYP2D6',
  'CYP1A2',
  'CYP1A1',
  'CYP2C19',
  'CYP2C8',
  'CYP2C9',
  'CYP3A5',
  'UGT1A1'],
 ['CYP', 'CYP3A4', 'CYP2D6', 'CYP1A1', 'CYP1A2'],
 ['CYP',
  'CYP2D6',
  'CYP3A4',
  'CYP1A2',
  'CYP1A1',
  'CYP2C8',
  'CYP2C19',
  'CYP2A6',
  'CYP2C9',
  'CYP2B6',
  'CYP1A',
  'CYP3A5'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'UGT1A1',
  'CYP1A2',
  'UGT1A9',
  'CYP2C19',
  'UGT1A3',
  'CYP2C9',
  'UGT1A10',
  'CYP2B6',
  'CYP2D6',
  'CYP1A1',
  'CYP2A6',
  'CYP3A5'],
 ['CYP',
  'CYP3A4',
  'CYP1A1',
  'CYP1A2',
  'CYP2D6',
  'CYP2C8',
  'CYP2C19',
  'CYP3A5',
  'CYP2C9'

In [46]:
import json

# 保存为 JSON 文件
with open('combined_enzyme.json', 'w', encoding='utf-8') as f:
    json.dump(combined_list, f, ensure_ascii=False, indent=4)

In [47]:
import json

# Load JSON data from files
def load_json_data(filepath):
    with open(filepath, "r") as file:
        return json.load(file)

# Save data to a new JSON file
def save_json_data(data, filepath):
    with open(filepath, "w") as file:
        json.dump(data, file)

# Function to find a metabolite matching by enzyme name or prefix
def find_metabolite_by_prefix(enzyme, metabolite_list):#, prefixes):
    matches = []
    flag=False
    # Try to find an exact match first
    for metabolite in metabolite_list:
        if enzyme == metabolite[0]:
            matches.append(canonicalize_smiles(metabolite[1]))
            flag=True
    if flag==True:
        return '|'.join(matches) if matches else None
    # If no exact match, try to match by known prefixes

        # Check if any metabolite starts with the same prefix
    for metabolite in metabolite_list:
        if metabolite[0].startswith(enzyme):
            matches.append(canonicalize_smiles(metabolite[1]))
    return '|'.join(list(set(matches))) if matches else None  # Return None if no match found

# Enzyme prefixes for matching
# enzyme_prefixes = ['CYP', 'UGT', 'SULT', 'AKR', 'GST', 'CES', 'FMO', 'ALDH', 'NAT', 'COMT', 'AOX', 'XDH', 'EPHX', 'NQO', 'ADH']

# Load both datasets
enzymes_data = load_json_data("combined_enzyme.json")
metabolites_data = load_json_data("combined_list.json")

# Check if the lists are of the correct length and structure
if len(enzymes_data) == 51 and len(metabolites_data) == 51:
    combined_data = []
    for enzyme_list, metabolite_list in zip(enzymes_data, metabolites_data):
        combined_group = []
        for enzyme in enzyme_list:
            # Find matching metabolite by enzyme or prefix
            metabolite = find_metabolite_by_prefix(enzyme, metabolite_list)#, enzyme_prefixes)
            combined_group.append((enzyme, metabolite))
        combined_data.append(combined_group)

    save_json_data(combined_data, "path_to_updated_combined_data.json")
else:
    print("Error: Data lists do not match the required structure or length.")


In [48]:
for i in range(len(combined_list)):
    combined_list[i] = combined_list[i] + [""] * (20 - len(combined_list[i]))

In [49]:
# num=65
all=[]
for i in range(num):
    data=[]
    for k in range(20):     
        data.append(combined_list[i][k])
    all.append("|".join(data))

In [50]:
all

['CYP|UGT|CYP3A4|CYP2C9|CYP2C19|CYP1A2|CYP2D6|CYP3A5|CYP1A1|CYP2B6|UGT1A9|UGT1A1|CYP2C8|UGT1A3||||||',
 'CYP|UGT|CYP3A4|UGT1A9|CYP2C8|UGT2B7|UGT1A1|UGT1A8|UGT2B4|CYP2D6|CYP2C19|CYP1A1||||||||',
 'CYP|CYP2C19|CYP2D6|CYP3A4|CYP1A2|CYP2C9|CYP2C8|CYP1A1||||||||||||',
 'CYP|UGT|CYP3A4|CYP2D6|CYP1A2|CYP1A1|CYP2C19|CYP2C8|CYP2C9|CYP3A5|UGT1A1|||||||||',
 'CYP|CYP3A4|CYP2D6|CYP1A1|CYP1A2|||||||||||||||',
 'CYP|CYP2D6|CYP3A4|CYP1A2|CYP1A1|CYP2C8|CYP2C19|CYP2A6|CYP2C9|CYP2B6|CYP1A|CYP3A5||||||||',
 'CYP|UGT|CYP3A4|UGT1A1|CYP1A2|UGT1A9|CYP2C19|UGT1A3|CYP2C9|UGT1A10|CYP2B6|CYP2D6|CYP1A1|CYP2A6|CYP3A5|||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP2D6|CYP2C8|CYP2C19|CYP3A5|CYP2C9|CYP1B1||||||||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP3A5|CYP2D6|CYP2C9|CYP2C19||||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP2C8|CYP1A1|CYP2D6|CYP2A6|CYP2C9||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP1A1|CYP2C8|CYP2D6|CYP2C9|||||||||||',
 'UGT|CYP|AKR|UGT1A4|CYP3A4|UGT1A1|CYP2D6|AKR1C1|UGT1A3|AKR1C2||||||||||',
 'CES|CYP|UG

In [47]:
# truth=test['metabolite']

In [48]:
# len(truth)

### Prediction

### Calculate top-k accuracy

In [51]:
def get_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    if mol1 and mol2:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
        return TanimotoSimilarity(fp1, fp2)
    return 0

In [52]:
def kekulize_smiles(smi):
    smi = smi.replace(' ','')
    mol = Chem.MolFromSmiles(smi)
    Chem.Kekulize(mol, clearAromaticFlags = True)
    smi = Chem.MolToSmiles(mol, canonical = False, kekuleSmiles = True)
    return smi

In [53]:
len(all)

51

In [54]:
all

['CYP|UGT|CYP3A4|CYP2C9|CYP2C19|CYP1A2|CYP2D6|CYP3A5|CYP1A1|CYP2B6|UGT1A9|UGT1A1|CYP2C8|UGT1A3||||||',
 'CYP|UGT|CYP3A4|UGT1A9|CYP2C8|UGT2B7|UGT1A1|UGT1A8|UGT2B4|CYP2D6|CYP2C19|CYP1A1||||||||',
 'CYP|CYP2C19|CYP2D6|CYP3A4|CYP1A2|CYP2C9|CYP2C8|CYP1A1||||||||||||',
 'CYP|UGT|CYP3A4|CYP2D6|CYP1A2|CYP1A1|CYP2C19|CYP2C8|CYP2C9|CYP3A5|UGT1A1|||||||||',
 'CYP|CYP3A4|CYP2D6|CYP1A1|CYP1A2|||||||||||||||',
 'CYP|CYP2D6|CYP3A4|CYP1A2|CYP1A1|CYP2C8|CYP2C19|CYP2A6|CYP2C9|CYP2B6|CYP1A|CYP3A5||||||||',
 'CYP|UGT|CYP3A4|UGT1A1|CYP1A2|UGT1A9|CYP2C19|UGT1A3|CYP2C9|UGT1A10|CYP2B6|CYP2D6|CYP1A1|CYP2A6|CYP3A5|||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP2D6|CYP2C8|CYP2C19|CYP3A5|CYP2C9|CYP1B1||||||||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP3A5|CYP2D6|CYP2C9|CYP2C19||||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP2C8|CYP1A1|CYP2D6|CYP2A6|CYP2C9||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP1A1|CYP2C8|CYP2D6|CYP2C9|||||||||||',
 'UGT|CYP|AKR|UGT1A4|CYP3A4|UGT1A1|CYP2D6|AKR1C1|UGT1A3|AKR1C2||||||||||',
 'CES|CYP|UG

In [61]:
df=pd.read_csv("new_predict_add_true_drugbank_47_51.csv")

In [62]:
df

,predict_enzymes,predict_metabolites,substrate_smiles,true_label,true_enzymes,new_output
0,CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP...,CYP1A2|Cc1ncc2n1-c1cc(O)c(Cl)cc1C(c1ccccc1F)=N...,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2O,CYP3A4|CYP3A5|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc...,CYP3A4|CYP3A5|UGT1A4|UGT2B4|UGT2B7,CYP3A4|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2...
1,UGT|SULT,UGT|O=C(O)C1OC(O[C@H]2[C@H](c3ccc(Cl)c(Cc4ccc(...,OC[C@H]1O[C@@H](C2=CC=C(Cl)C(CC3=CC=C(OCCOC4CC...,UGT1A1|UGT1A9|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)...,CYP2C8|CYP2D6|CYP3A4|UGT1A1|UGT1A9,UGT1A1|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)[C@H](c...
2,UGT|SULT,UGT|COc1cc2c(cc1OC)[C@H]1C[C@@H](OC3OC(C(=O)O)...,COC1=CC2=C(C=C1OC)[C@H]1C[C@@H](O)[C@H](CC(C)C...,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP2D6,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...
3,UGT|SULT|EC 2.3.1.13|EC 2.3.1.7,UGT|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](OC2OC(C(=O...,COC1=CC(C)=C2NC=CC2=C1CN1CC[C@H](O)C[C@H]1C1=C...,UGT1A1|UGT1A3|UGT1A8|COc1cc(C)c2[nH]ccc2c1CN1C...,UGT1A1|UGT1A3|UGT1A8,UGT1A1|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H...
4,AOX|CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19...,AOX|<separated>AOX|CC1(C)CC(=O)N(CCCCN2CCN(c3n...,CC1(C)CC(=O)N(CCCCN2CCN(C3=NC=CC=N3)CC2)C(=O)C1,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP3A4,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...
5,EC 4.2.1.84|EC 3.5.5.1,EC 4.2.1.84|CN1CC[C@@H](NC(=O)Nc2ccc(C(N)=O)cc...,CN1CC[C@@H](NC(=O)NC2=CC=C(C#N)C=C2)C[C@@H]1C1...,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP3A4,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...
6,EC 3.5.2.X|UGT,EC 3.5.2.X|CNc1ccc(Cl)cc1N(C(=O)CC(=O)O)c1ccc(...,CN1C(=O)CC(=O)N(C2=CC=C(O)C=C2)C2=CC(Cl)=CC=C12,CYP2B6|CYP2C9|CYP3A4|O=C1CC(=O)N(c2ccc(O)cc2)c...,CYP2B6|CYP2C9|CYP3A4,CYP2B6|O=C1CC(=O)N(c2ccc(O)cc2)c2cc(Cl)ccc2N1<...
7,ADH|UGT|SULT|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP2...,ADH|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2C...,CC1=NC(NC2=NC=C(C(=O)NC3=C(C)C=CC=C3Cl)S2)=CC(...,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP3A4,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...
8,CYP1A2|CYP2E1|CYP2A6|CYP2C9|CYP2C19|CYP2D6|CYP...,CYP1A2|CCOc1ccc(S(=O)(=O)N2CCN(CC)CC2)cc1-c1nn...,CCCC1=NC(C)=C2C(=O)N=C(C3=CC(S(=O)(=O)N4CCN(CC...,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP3A4,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...
9,CYP1A2|CYP2A6|CYP2B6|AOX|CYP2C8|CYP2C9|CYP2C19...,CYP1A2|FC1=CC=C(C=C1)C2(OCC3=CC(C#N)=C(C=C32)O...,N#CC1=CC=C2C(=C1)COC2(CCC=O)C1=CC=C(F)C=C1,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,AOX,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1


In [63]:
df['MetePred_predict']=all


def process_predict(predict):
    # 1. 先用 '|' 拆分字符串
    parts = predict.split('|')
    
    # 2. 过滤掉空字符串
    non_empty_parts = [part for part in parts if part]
    
    # 3. 使用 '|' 重新连接非空部分
    return '|'.join(non_empty_parts)

# 对 df 的 predict 列进行处理
df['MetePred_predict'] = df['MetePred_predict'].apply(process_predict)




In [67]:
df

,predict_enzymes,predict_metabolites,substrate_smiles,true_label,true_enzymes,new_output,MetePred_predict,MetePred_metabolite
0,CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP...,CYP1A2|Cc1ncc2n1-c1cc(O)c(Cl)cc1C(c1ccccc1F)=N...,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2O,CYP3A4|CYP3A5|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc...,CYP3A4|CYP3A5|UGT1A4|UGT2B4|UGT2B7,CYP3A4|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2...,CYP|UGT|CYP3A4|CYP2C9|CYP2C19|CYP1A2|CYP2D6|CY...,CYP|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O|C...
1,UGT|SULT,UGT|O=C(O)C1OC(O[C@H]2[C@H](c3ccc(Cl)c(Cc4ccc(...,OC[C@H]1O[C@@H](C2=CC=C(Cl)C(CC3=CC=C(OCCOC4CC...,UGT1A1|UGT1A9|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)...,CYP2C8|CYP2D6|CYP3A4|UGT1A1|UGT1A9,UGT1A1|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)[C@H](c...,CYP|UGT|CYP3A4|UGT1A9|CYP2C8|UGT2B7|UGT1A1|UGT...,CYP|OC[C@H]1O[C@@H](c2ccc(Cl)c(Cc3ccc(O)cc3)c2...
2,UGT|SULT,UGT|COc1cc2c(cc1OC)[C@H]1C[C@@H](OC3OC(C(=O)O)...,COC1=CC2=C(C=C1OC)[C@H]1C[C@@H](O)[C@H](CC(C)C...,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP2D6,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP|CYP2C19|CYP2D6|CYP3A4|CYP1A2|CYP2C9|CYP2C8...,CYP|COc1cc2c(cc1OC)[C@H]1C[C@@H](O)[C@H](CC(C)...
3,UGT|SULT|EC 2.3.1.13|EC 2.3.1.7,UGT|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](OC2OC(C(=O...,COC1=CC(C)=C2NC=CC2=C1CN1CC[C@H](O)C[C@H]1C1=C...,UGT1A1|UGT1A3|UGT1A8|COc1cc(C)c2[nH]ccc2c1CN1C...,UGT1A1|UGT1A3|UGT1A8,UGT1A1|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H...,CYP|UGT|CYP3A4|CYP2D6|CYP1A2|CYP1A1|CYP2C19|CY...,CYP|COc1cc(CO)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H]1...
4,AOX|CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19...,AOX|<separated>AOX|CC1(C)CC(=O)N(CCCCN2CCN(c3n...,CC1(C)CC(=O)N(CCCCN2CCN(C3=NC=CC=N3)CC2)C(=O)C1,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP3A4,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP|CYP3A4|CYP2D6|CYP1A1|CYP1A2,CYP|c1cnc(N2CCNCC2)nc1<separated>CYP3A4|c1cnc(...
5,EC 4.2.1.84|EC 3.5.5.1,EC 4.2.1.84|CN1CC[C@@H](NC(=O)Nc2ccc(C(N)=O)cc...,CN1CC[C@@H](NC(=O)NC2=CC=C(C#N)C=C2)C[C@@H]1C1...,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP3A4,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP|CYP2D6|CYP3A4|CYP1A2|CYP1A1|CYP2C8|CYP2C19...,CYP|CN1CCC(NC(=O)Nc2ccc(C#N)cc2)CC1c1nc2cc(O)c...
6,EC 3.5.2.X|UGT,EC 3.5.2.X|CNc1ccc(Cl)cc1N(C(=O)CC(=O)O)c1ccc(...,CN1C(=O)CC(=O)N(C2=CC=C(O)C=C2)C2=CC(Cl)=CC=C12,CYP2B6|CYP2C9|CYP3A4|O=C1CC(=O)N(c2ccc(O)cc2)c...,CYP2B6|CYP2C9|CYP3A4,CYP2B6|O=C1CC(=O)N(c2ccc(O)cc2)c2cc(Cl)ccc2N1<...,CYP|UGT|CYP3A4|UGT1A1|CYP1A2|UGT1A9|CYP2C19|UG...,CYP|CN1C(=O)CC(=O)N(c2ccc(O)c(O)c2)c2cc(Cl)ccc...
7,ADH|UGT|SULT|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP2...,ADH|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2C...,CC1=NC(NC2=NC=C(C(=O)NC3=C(C)C=CC=C3Cl)S2)=CC(...,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP3A4,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP|CYP3A4|CYP1A1|CYP1A2|CYP2D6|CYP2C8|CYP2C19...,CYP|C=C1C=CC=C(Cl)/C1=N\C(=O)c1cnc(Nc2cc(N3CCN...
8,CYP1A2|CYP2E1|CYP2A6|CYP2C9|CYP2C19|CYP2D6|CYP...,CYP1A2|CCOc1ccc(S(=O)(=O)N2CCN(CC)CC2)cc1-c1nn...,CCCC1=NC(C)=C2C(=O)N=C(C3=CC(S(=O)(=O)N4CCN(CC...,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP3A4,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP|CYP3A4|CYP1A1|CYP1A2|CYP3A5|CYP2D6|CYP2C9|...,CYP|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N(CC=O)...
9,CYP1A2|CYP2A6|CYP2B6|AOX|CYP2C8|CYP2C9|CYP2C19...,CYP1A2|FC1=CC=C(C=C1)C2(OCC3=CC(C#N)=C(C=C32)O...,N#CC1=CC=C2C(=C1)COC2(CCC=O)C1=CC=C(F)C=C1,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,AOX,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP2C8|CYP1A1...,CYP|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1<se...


In [64]:
formatted_results = []
for data in combined_data:
    formatted_data = "<separated>".join([f"{enzyme}|{metabolite}" for enzyme, metabolite in data])
    formatted_results.append(formatted_data)

# 输出所有格式化后的结果
for result in formatted_results:
    print(result)

CYP|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O|Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12<separated>UGT|Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(=O)O)[C@@H](O)[C@H](O)[C@H]1O<separated>CYP3A4|Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12<separated>CYP2C9|Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12<separated>CYP2C19|Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12<separated>CYP1A2|Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12<separated>CYP2D6|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O<separated>CYP3A5|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O<separated>CYP1A1|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O<separated>CYP2B6|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O<separated>UGT1A9|Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(=O)O)[C@@H](O)[C@H](O)[C@H]1O<separated>UGT1A1|Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(=O)O)[C@@H](O)[C@H](O)[C@H]1O<separated>CYP2C8|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O<separated>UGT1A3|Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(=O)O)[C@@H](O)[C@H](O)[C@H]1O
CYP|OC[C@H]1O[C@@

In [65]:
df['MetePred_metabolite']=formatted_results

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

# 定义计算相似度的函数
def get_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    if mol1 and mol2:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
        return TanimotoSimilarity(fp1, fp2)
    return 0

# 定义函数，计算酶匹配后代谢物的正确率
def calculate_metabolite_accuracy(df, top):
    # 初始化统计指标
    at_least_one = 0
    at_least_half = 0
    all_metabolites = 0
    total_rows = len(df)
    len_corr_preds, len_preds, len_actual_metabolites = 0, 0, 0
    
    for idx, row in df.iterrows():
        # 分割 output 列中的酶和代谢物
        if '<separated>' in row['new_output']:
            output_pairs = row['new_output'].split('<separated>')
        else:
            output_pairs = [row['new_output']]  # 否则将整个 output 作为单个对处理
        
        # 分割 predict_metabolite 列中的酶和代谢物，并只取前 top 个预测代谢物
        if '<separated>' in row['MetePred_metabolite']:
            predictions = row['MetePred_metabolite'].split('<separated>')[:top]
        else:
            predictions = [row['MetePred_metabolite']]  # 否则将整个 predict_metabolite 作为单个对处理
        
        total_pairs = len(output_pairs)
        corr_preds = []  # 存储匹配上的预测代谢物
        len_actual_metabolites += total_pairs  # 记录真实代谢物的总数
        
        for output_pair in output_pairs:
            if '|' not in output_pair:
                continue
            output_enzyme, output_metabolite = output_pair.split('|', 1)

            for pred in predictions:
                if '|' not in pred:
                    continue
                pred_enzyme, pred_metabolite = pred.split('|', 1)

                # 如果酶匹配，则比较代谢物
                if pred_enzyme == output_enzyme:
                    similarity = get_similarity(output_metabolite, pred_metabolite)
                    if similarity == 1:  # 相似度阈值
                        corr_preds.append(pred)
                        break  # 匹配后跳出内循环，避免重复匹配

        # 统计匹配情况
        len_corr_preds += len(corr_preds)  # 匹配到的代谢物数
        len_preds += len(predictions)  # 预测的代谢物总数

        # 根据匹配情况更新统计指标
        if len(corr_preds) > 0:
            at_least_one += 1  # 至少有一个匹配
        if len(corr_preds) >= total_pairs / 2:
            at_least_half += 1  # 至少匹配一半
        if len(corr_preds) == total_pairs:
            all_metabolites += 1  # 全部匹配成功

    precision = len_corr_preds / len_preds if len_preds > 0 else 0
    recall = len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0
    
    # 计算 F1 score
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    # 保存结果到文件
    with open("results_enzyme_accuracy_by_top_new.log", 'a') as f:
        f.write(
            ",".join([
                str("top"+str(top)), 
                "{:.2%}".format(at_least_one / total_rows),  # 至少匹配一个的比例
                "{:.2%}".format(at_least_half / total_rows),  # 至少匹配一半的比例
                "{:.2%}".format(all_metabolites / total_rows),  # 全部匹配的比例
                str(len_corr_preds),  # 匹配正确的代谢物数量
                str(len_preds),  # 总预测代谢物数量
                str(len_actual_metabolites),  # 总实际代谢物数量
                "{:.2%}".format(len_corr_preds / len_preds if len_preds > 0 else 0),  # 精确率
                "{:.2%}".format(len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0),  # 召回率
                "{:.2%}".format(f1_score)    # F1 score
            ]) + "\n"
        )



# 计算不同 top 值的代谢物匹配的准确性
top_values = [5, 10, 15, 20]

for top in top_values:
    calculate_metabolite_accuracy(df, top)

print("计算完成并已保存到 results_enzyme_accuracy_by_top.log 文件")


计算完成并已保存到 results_enzyme_accuracy_by_top.log 文件


In [69]:
df.to_csv("my_predict_enzyme_metabolite.csv",index=False)

In [1]:
import pandas as pd
df=pd.read_csv("my_predict_enzyme_metabolite.csv")

## if you have true labels, then can calculate the metrics

In [72]:
df

,predict_enzymes,predict_metabolites,substrate_smiles,true_label,true_enzymes,new_output,MetePred_predict,MetePred_metabolite
0,CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP...,CYP1A2|Cc1ncc2n1-c1cc(O)c(Cl)cc1C(c1ccccc1F)=N...,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2O,CYP3A4|CYP3A5|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc...,CYP3A4|CYP3A5|UGT1A4|UGT2B4|UGT2B7,CYP3A4|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2...,CYP|UGT|CYP3A4|CYP2C9|CYP2C19|CYP1A2|CYP2D6|CY...,CYP|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O|C...
1,UGT|SULT,UGT|O=C(O)C1OC(O[C@H]2[C@H](c3ccc(Cl)c(Cc4ccc(...,OC[C@H]1O[C@@H](C2=CC=C(Cl)C(CC3=CC=C(OCCOC4CC...,UGT1A1|UGT1A9|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)...,CYP2C8|CYP2D6|CYP3A4|UGT1A1|UGT1A9,UGT1A1|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)[C@H](c...,CYP|UGT|CYP3A4|UGT1A9|CYP2C8|UGT2B7|UGT1A1|UGT...,CYP|OC[C@H]1O[C@@H](c2ccc(Cl)c(Cc3ccc(O)cc3)c2...
2,UGT|SULT,UGT|COc1cc2c(cc1OC)[C@H]1C[C@@H](OC3OC(C(=O)O)...,COC1=CC2=C(C=C1OC)[C@H]1C[C@@H](O)[C@H](CC(C)C...,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP2D6,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP|CYP2C19|CYP2D6|CYP3A4|CYP1A2|CYP2C9|CYP2C8...,CYP|COc1cc2c(cc1OC)[C@H]1C[C@@H](O)[C@H](CC(C)...
3,UGT|SULT|EC 2.3.1.13|EC 2.3.1.7,UGT|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](OC2OC(C(=O...,COC1=CC(C)=C2NC=CC2=C1CN1CC[C@H](O)C[C@H]1C1=C...,UGT1A1|UGT1A3|UGT1A8|COc1cc(C)c2[nH]ccc2c1CN1C...,UGT1A1|UGT1A3|UGT1A8,UGT1A1|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H...,CYP|UGT|CYP3A4|CYP2D6|CYP1A2|CYP1A1|CYP2C19|CY...,CYP|COc1cc(CO)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H]1...
4,AOX|CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19...,AOX|<separated>AOX|CC1(C)CC(=O)N(CCCCN2CCN(c3n...,CC1(C)CC(=O)N(CCCCN2CCN(C3=NC=CC=N3)CC2)C(=O)C1,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP3A4,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP|CYP3A4|CYP2D6|CYP1A1|CYP1A2,CYP|c1cnc(N2CCNCC2)nc1<separated>CYP3A4|c1cnc(...
5,EC 4.2.1.84|EC 3.5.5.1,EC 4.2.1.84|CN1CC[C@@H](NC(=O)Nc2ccc(C(N)=O)cc...,CN1CC[C@@H](NC(=O)NC2=CC=C(C#N)C=C2)C[C@@H]1C1...,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP3A4,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP|CYP2D6|CYP3A4|CYP1A2|CYP1A1|CYP2C8|CYP2C19...,CYP|CN1CCC(NC(=O)Nc2ccc(C#N)cc2)CC1c1nc2cc(O)c...
6,EC 3.5.2.X|UGT,EC 3.5.2.X|CNc1ccc(Cl)cc1N(C(=O)CC(=O)O)c1ccc(...,CN1C(=O)CC(=O)N(C2=CC=C(O)C=C2)C2=CC(Cl)=CC=C12,CYP2B6|CYP2C9|CYP3A4|O=C1CC(=O)N(c2ccc(O)cc2)c...,CYP2B6|CYP2C9|CYP3A4,CYP2B6|O=C1CC(=O)N(c2ccc(O)cc2)c2cc(Cl)ccc2N1<...,CYP|UGT|CYP3A4|UGT1A1|CYP1A2|UGT1A9|CYP2C19|UG...,CYP|CN1C(=O)CC(=O)N(c2ccc(O)c(O)c2)c2cc(Cl)ccc...
7,ADH|UGT|SULT|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP2...,ADH|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2C...,CC1=NC(NC2=NC=C(C(=O)NC3=C(C)C=CC=C3Cl)S2)=CC(...,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP3A4,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP|CYP3A4|CYP1A1|CYP1A2|CYP2D6|CYP2C8|CYP2C19...,CYP|C=C1C=CC=C(Cl)/C1=N\C(=O)c1cnc(Nc2cc(N3CCN...
8,CYP1A2|CYP2E1|CYP2A6|CYP2C9|CYP2C19|CYP2D6|CYP...,CYP1A2|CCOc1ccc(S(=O)(=O)N2CCN(CC)CC2)cc1-c1nn...,CCCC1=NC(C)=C2C(=O)N=C(C3=CC(S(=O)(=O)N4CCN(CC...,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP3A4,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP|CYP3A4|CYP1A1|CYP1A2|CYP3A5|CYP2D6|CYP2C9|...,CYP|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N(CC=O)...
9,CYP1A2|CYP2A6|CYP2B6|AOX|CYP2C8|CYP2C9|CYP2C19...,CYP1A2|FC1=CC=C(C=C1)C2(OCC3=CC(C#N)=C(C=C32)O...,N#CC1=CC=C2C(=C1)COC2(CCC=O)C1=CC=C(F)C=C1,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,AOX,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP2C8|CYP1A1...,CYP|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1<se...


In [73]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

top=[5,10,15,20]
for top_i in top:
    corr = 0 
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0

    # 定义相似度函数

    # 遍历每一行
    for i in range(len(df)):
        # substrate = df.iloc[i]['substrate']
        true_metabolites = df.iloc[i]['true_enzymes'].strip().split('|')
        original_preds = df.iloc[i]['MetePred_predict'].strip().split('|')[:top_i]
        # original_preds = df.iloc[i][f"{top}_new"].strip().split('|')
        
        unique_preds = list(set(original_preds))

        original_preds = unique_preds  # 去重后的预测值
        len_preds += len(original_preds)

        corr_preds = []
        # preds_to_remove = []
        
        for true_metabolite in true_metabolites:
            for pred in original_preds:
                if pred==true_metabolite:
                    corr_preds.append(pred)
                    # preds_to_remove.append(pred)
                    break  # 跳出内层循环，防止同一预测值多次计入

        # 移除已匹配的预测值，确保每个预测值只匹配一次
        # original_preds = [pred for pred in original_preds if pred not in preds_to_remove]
        
        if len(corr_preds) > 0:
            at_least_one += 1
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)

    with open("results_enzyme_unique.log", 'a') as f:
        f.write(
            ",".join([
                str("top"+str(top_i)), 
                "{:.2%}".format(at_least_one / len(df)), # at_least_one_metabolite
                "{:.2%}".format(at_least_half / len(df)), # at_least_half_metabolites
                "{:.2%}".format(all_metabolites / len(df)), # all_metabolites
                str(len_corr_preds),  # correct_predictions
                str(len_preds),  # total_predictions
                str(len_actual_metabolites),
                "{:.2%}".format(len_corr_preds / len_preds), # precision
                "{:.2%}".format(len_corr_preds / len_actual_metabolites), # recall
            ]) + "\n"
        )



In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

# 定义相似度函数
def get_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    if mol1 and mol2:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
        return TanimotoSimilarity(fp1, fp2)
    return 0

# 假设 df 是你的数据框
top = [5, 10, 13, 20]
for top_i in top:
    corr = 0 
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0
    no_correct_predictions_indices = []  # 记录没有正确预测的行的索引

    # 遍历每一行
    for i in range(len(df)):
        substrate = df.iloc[i]['substrate']
        true_metabolites = df.iloc[i]['metabolite'].strip().split('|')
        original_preds = df.iloc[i]['predict'].strip().split('|')[:top_i]
        expanded_preds = []
        for pred in original_preds:
            expanded_preds.extend(pred.split('.'))
        original_preds = expanded_preds
        
        unique_preds = list(set(original_preds))

        original_preds = unique_preds  # 去重后的预测值
        len_preds += len(original_preds)

        corr_preds = []
        preds_to_remove = []
        
        for true_metabolite in true_metabolites:
            for pred in original_preds:
                try:
                    pred = Chem.MolToSmiles(Chem.MolFromSmiles(pred))
                except:
                    print(pred)
                    invalid += 1
                    continue
                if get_similarity(pred, true_metabolite) == 1:
                    corr_preds.append(pred)
                    preds_to_remove.append(pred)
                    break  # 跳出内层循环，防止同一预测值多次计入

        # 移除已匹配的预测值，确保每个预测值只匹配一次
        original_preds = [pred for pred in original_preds if pred not in preds_to_remove]
        
        if len(corr_preds) > 0:
            at_least_one += 1
        else:
            no_correct_predictions_indices.append(i)  # 记录没有正确预测的行的索引
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)

    # with open("results.log", 'a') as f:
    #     f.write(
    #         ",".join([
    #             str("top"+str(top_i)), 
    #             "{:.2%}".format(at_least_one / len(df)),  # at_least_one_metabolite
    #             "{:.2%}".format(at_least_half / len(df)),  # at_least_half_metabolites
    #             "{:.2%}".format(all_metabolites / len(df)),  # all_metabolites
    #             "{:.2%}".format(len_corr_preds / len_preds),  # precision
    #             "{:.2%}".format(len_corr_preds / len_actual_metabolites),  # recall
    #         ]) + "\n"
    #     )

    # 输出没有正确预测的行的索引
    print(f"Top {top_i}: 没有任何正确预测的行的索引: {no_correct_predictions_indices}")


In [ ]:
# Top 5: 没有任何正确预测的行的索引: [37, 40, 43, 54, 56, 62, 64]
# Top 10: 没有任何正确预测的行的索引: [37, 40, 43, 56, 62]
# Top 13: 没有任何正确预测的行的索引: [37, 40, 43, 62]
# Top 20: 没有任何正确预测的行的索引: [40, 62]

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

# 定义相似度函数
def get_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    if mol1 and mol2:
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
        return TanimotoSimilarity(fp1, fp2)
    return 0

# 假设 df 是你的数据框
top = [5, 10, 13, 20]
for top_i in top:
    corr = 0 
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0
    no_correct_predictions_indices = []  # 记录没有正确预测的行的索引

    # 遍历每一行
    for i in range(len(df)):
        substrate = df.iloc[i]['substrate']
        true_metabolites = df.iloc[i]['metabolite'].strip().split('|')
        original_preds = df.iloc[i]['predict'].strip().split('|')[:top_i]
        expanded_preds = []
        preds=[]
        # original_preds=predict_smiles_dict[original_preds_short]
        for pred in original_preds:
            # print(pred)
            # if pred=='O=P1(NCCCl)NCCCO1':
            pred=predict_smiles_dict.get(pred, pred)
            print(pred)
            preds.extend([pred])
            expanded_preds.extend(pred.split('.'))
            # print(expanded_preds)
        df.at[i, 'predict_all'] = "|".join(preds)
        original_preds = expanded_preds
        
        unique_preds = list(set(original_preds))

        original_preds = unique_preds  # 去重后的预测值
        # print(original_preds)
        len_preds += len(original_preds)

        corr_preds = []
        preds_to_remove = []
        
        for true_metabolite in true_metabolites:
            for pred in original_preds:
                try:
                    pred = Chem.MolToSmiles(Chem.MolFromSmiles(pred))
                except:
                    print(pred)
                    invalid += 1
                    continue
                if get_similarity(pred, true_metabolite) == 1:
                    corr_preds.append(pred)
                    preds_to_remove.append(pred)
                    break  # 跳出内层循环，防止同一预测值多次计入

        # 移除已匹配的预测值，确保每个预测值只匹配一次
        original_preds = [pred for pred in original_preds if pred not in preds_to_remove]
        
        if len(corr_preds) > 0:
            at_least_one += 1
        else:
            no_correct_predictions_indices.append(i)  # 记录没有正确预测的行的索引
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)

    with open("results.log", 'a') as f:
        f.write(
            ",".join([
                str("top"+str(top_i)), 
                "{:.2%}".format(at_least_one / len(df)),  # at_least_one_metabolite
                "{:.2%}".format(at_least_half / len(df)),  # at_least_half_metabolites
                "{:.2%}".format(all_metabolites / len(df)),  # all_metabolites
                str(len_corr_preds),  # correct_predictions
                str(len_preds),  # total_predictions
                str(len_actual_metabolites),
                "{:.2%}".format(len_corr_preds / len_preds),  # precision
                "{:.2%}".format(len_corr_preds / len_actual_metabolites),  # recall
            ]) + "\n"
        )

    # 输出没有正确预测的行的索引
    print(f"Top {top_i}: 没有任何正确预测的行的索引: {no_correct_predictions_indices}")


In [ ]:
pred

In [ ]:
preds=[]

In [ ]:
preds.extend(pred)

In [ ]:
preds

In [ ]:
predict_smiles_dict['O=P1(NCCCl)NCCCO1']

In [ ]:
O=CCCl

In [ ]:
df

In [ ]:
df.to_csv("predict_all_small_new_add_new11.csv",index=False)

In [ ]:
df.iloc[56]

In [ ]:
df.iloc[62]

In [ ]:
# calculate the accuracy of predicted smiles
#     for pred in original_preds:
#         expanded_preds.extend(pred.split('.'))
# print("Top-K Pred_Set Prediction: ")
k_lis = []
for i in range(num):
    rank = 21
    for k in range(20):
        
        if get_similarity(pre_smiles_50_lis[i][k],truth[i])==1:
            rank = k+1
            break
    k_lis.append(rank)


top20 = 0
top5 = 0
top10 = 0
for i in k_lis:
    if i <= 5:
        top5 += 1
    if i <= 10:
        top10 += 1
    if i <= 20:
        top20 += 1



print(f'Top-5 Pred accuracy after TTA: {top5/num:.4f}')
print(f'Top-10 Pred accuracy after TTA: {top10/num:.4f}')
print(f'Top-20 Pred accuracy after TTA: {top20/num:.4f}')

In [ ]:
# calculate the accuracy of predicted smiles
#     for pred in original_preds:
#         expanded_preds.extend(pred.split('.'))
# print("Top-K Pred_Set Prediction: ")
k_lis = []
for i in range(num):
    rank = 21
    for k in range(20):
        
        if get_similarity(pre_smiles_50_lis[i][k],truth[i])==1:
            rank = k+1
            break
    k_lis.append(rank)


top20 = 0
top5 = 0
top10 = 0
for i in k_lis:
    if i <= 5:
        top5 += 1
    if i <= 10:
        top10 += 1
    if i <= 20:
        top20 += 1



print(f'Top-5 Pred accuracy after TTA: {top5/num:.4f}')
print(f'Top-10 Pred accuracy after TTA: {top10/num:.4f}')
print(f'Top-20 Pred accuracy after TTA: {top20/num:.4f}')

In [ ]:
raw=pd.read_csv("/home/wangyitian/opennmt-model/output/test_set/esmiles/results/neibu/Raw_smiles_1times/predicted/output_step_1000000",sep='\t')

### Calculate top-k accuracy for each reaction type

In [ ]:
class_lis = []
with open('datasets/50k_ReactSeq/aug20_test/2023_7_13_50k_test_r20_v.txt') as f:
    for line in f.readlines():
        class_lis.append(line.split(',')[1])
class_lis = class_lis[:5000]

In [ ]:
for c in range(1,11):
    c ='class_' + str(c)
    num = 0
    top1 = 0
    top3 = 0
    top5 = 0
    top10 = 0
    for i, class_ in zip(k_lis,class_lis):
        if class_ == c:
            num += 1
            if i <= 1:
                top1 += 1
            if i <= 3:
                top3 += 1
            if i <= 5:
                top5 += 1
            if i <= 10:
                top10 += 1
        else:
            pass
    print(c,num)
    print(round(top1/num,4))
    print(round(top3/num,4))
    print(round(top5/num,4))
    print(round(top10/num,4))
    print('')

### Calculate top-k accuracy for each edit time

In [ ]:
edit_times_lis = []
for i in range(5000):
    core_edits,chai_edits,stereo_edits,charge_edits,core_edits_add,lg_map_lis = get_edit_from_e_smiles(true_reactseq_lis[i])
    edit_times = len(core_edits) + len(chai_edits) + len(stereo_edits) + len(charge_edits) + len(core_edits_add) + len(lg_map_lis)
    edit_times_lis.append(edit_times)

In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 1
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time == e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 2
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time == e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 3
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time == e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 4
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time == e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 5
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time == e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


In [ ]:
num = 0
top1 = 0
top3 = 0
top5 = 0
top10 = 0
e = 6
for i, edit_time in zip(k_lis,edit_times_lis):
    if edit_time >= e:
        num += 1
        if i <= 1:
            top1 += 1
        if i <= 3:
            top3 += 1
        if i <= 5:
            top5 += 1
        if i <= 10:
            top10 += 1
    else:
        pass
print(e,num,num/5000)
print(round(top1/num,4))
print(round(top3/num,4))
print(round(top5/num,4))
print(round(top10/num,4))
print('')


### Calculate top-k accuracy for Reaction Center Identification

In [ ]:
num = 5000

In [ ]:
ture_reactseq_lis = []
with open('datasets/50k_ReactSeq/aug20_test/2023_7_13_50k_test_r20_v.txt') as f:
    for line in f.readlines():
        ture_reactseq_lis.append(','.join(line.split(',')[2:]).replace('\n',''))

In [ ]:
true_reactseq_lis_g = [[ture_reactseq_lis[i + j*num] for j in range(20)] for i in range(num)]

In [ ]:
len(true_reactseq_lis_g)

In [ ]:
pre_reactseq_50_lis =[[j.split('<')[0] for j in i] for i in pre_reactseq_50_lis]

In [ ]:
pre_reactseq_50_lis_1 = []
for reactseq_50 in pre_reactseq_50_lis_:
    reactseq_50_1 = []
    for reactseq in reactseq_50:
        if reactseq not in reactseq_50_1:
            reactseq_50_1.append(reactseq)
    if len(reactseq_50_1) < 10:
        reactseq_50_1 += ['']*(10 - len(reactseq_50_1))
    pre_reactseq_50_lis_1.append(reactseq_50_1)

In [ ]:
k_lis = []
for i in range(num):
    rank = 11
    for k in range(n):
        if pre_reactseq_50_lis_1[i][k] in [i.split('<')[0] for i in true_reactseq_lis_g[i]]:
            rank = k+1
            break
    k_lis.append(rank)
    
#k_lis

In [ ]:
top1 = 0
top2 = 0
top3 = 0
top5 = 0
top10 = 0
for i in k_lis:
    if i <= 1:
        top1 += 1
    if i <= 2:
        top2 += 1
    if i <= 3:
        top3 += 1
    if i <= 5:
        top5 += 1
    if i <= 10:
        top10 += 1
        

print(f'Top-1 Pred Reaction Center Identification accuracy after TTA: {top1/num:.3f}')
print(f'Top-2 Pred Reaction Center Identification accuracy after TTA: {top2/num:.3f}')
print(f'Top-3 Pred Reaction Center Identification accuracy after TTA: {top3/num:.3f}')
print(f'Top-5 Pred Reaction Center Identification accuracy after TTA: {top5/num:.3f}')
print(f'Top-10 Pred Reaction Center Identification accuracy after TTA: {top10/num:.3f}')